In [3]:
import snowflake.connector
import pandas as pd
import seaborn as sns
import plotly.express as px
import warnings
import numpy as np
import matplotlib.pyplot as plt
import os
from snowflake.connector.pandas_tools import write_pandas
from snowflake.connector.pandas_tools import pd_writer
# usr = os.environ['svc_user']
# pwd = os.environ['svc_pwd']

usr = 'SVC_PROMO'
pwd = 'nBrVtHzLh8dk2Kvv'

warnings.filterwarnings('ignore')

In [4]:
def sf_connect():
    con = snowflake.connector.connect(  
        account = 'chewy.us-east-1',
        user = usr,
        password=pwd,
        autocommit = False,
        database= 'EDLDB',
        warehouse = 'IT_WH') #EDLDB_DSE_DEV
    return con
    
def exec_qry_all(qry):
    con = sf_connect()
    cur = con.cursor()
    try:
        cur.execute(qry,timeout=3600)
        return cur.fetch_pandas_all()   
    except Exception as e:
        print(e)
    finally:
        cur.close()

In [56]:
qry = '''SELECT * FROM EDLDB.ECOM_SANDBOX.CTR_TEMP_SV_ZONE_ALL WHERE ZONE in ('zone-1','zone-2','zone-3','zone-4','zone-5','zone-6','zone-7','zone-8','zone-9','zone-10','zone-11','zone-12','zone-13','zone-14','zone-15','zone-16','zone-17','zone-18','zone-19','zone-20')'''
qry_all = '''SELECT * FROM EDLDB.ECOM_SANDBOX.CTR_TEMP_SV_ZONE_ALL'''
# dataset = exec_qry_all(qry)
# dataset.head()

In [63]:
dataset = exec_qry_all(qry_all)
dataset.head()

,SESSIONS_DATE_EST,CHEWY_FINANCIAL_REPORTING_PERIOD,DEVICE_CATEGORY,CMS_LEVEL,PLACEMENT_SLOT_UID,WIDGET_POSITION,WIDGET_TYPE,WIDGET_TYPE_VALUE,ZONE,WIDGET_NAME,...,ASSET_SEGMENT_VIEWS,WIDGET_SEGMENT_CLICKS,ASSET_SEGMENT_CLICKS,PAGE_IMPRESSIONS,ATTRIBUTED_REVENUE,ATTRIBUTED_UNITS,ATTRIBUTED_ORDER,AUTOSHIP_ATTRIBUTED_REVENUE,AUTOSHIP_ATTRIBUTED_UNITS,AUTOSHIP_ATTRIBUTED_ORDER
0,2022-04-03,2022-P02,desktop,widget,5282354705363668,20,banner,hero-banner-w-cta,None,heartgard-hero-banner-20,...,NaN,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0
1,2022-03-29,2022-P02,desktop,asset,None,None,nav-flyout,nav-flyout-brand-tiles,None,dog-popular-brands-brand,...,1.0,NaN,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0
2,2022-04-05,2022-P03,desktop,asset,None,None,nav-flyout,nav-flyout-brand-tiles,None,cat-popular-brands-brand,...,1.0,NaN,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0
3,2022-04-01,2022-P02,desktop,asset,None,None,nav-flyout,nav-flyout-brand-tiles,None,dog-popular-brands-brand,...,2.0,NaN,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0
4,2022-04-21,2022-P03,desktop,asset,None,None,nav-flyout,nav-flyout-brand-tiles,None,dog-popular-brands-brand,...,2.0,NaN,0.0,NaN,0.0,0.0,0.0,0.0,0.0,0.0


In [64]:
len(dataset)

30221792

---

---

In [59]:
def asset_scoring():
    #Filter by widget, date range and calculate CTR, UNIT/K
    data = dataset[dataset['CMS_LEVEL'] == 'asset']

    data['REV/K'] = ((data['ATTRIBUTED_REVENUE']/data['ASSET_SEGMENT_VIEWS'])*1000).round(2)
    data['REV/K'].replace([np.inf, -np.inf], np.nan, inplace=True)
    data['REV/K'] = data['REV/K'].fillna(0)

    end_date = data['SESSIONS_DATE_EST'].max()
    df_7D = data[(data['SESSIONS_DATE_EST'] >= (end_date - pd.to_timedelta(7, unit='d')))&(data['SESSIONS_DATE_EST'] <= end_date)]
    df_14D = data[(data['SESSIONS_DATE_EST'] >= (end_date - pd.to_timedelta(14, unit='d')))&(data['SESSIONS_DATE_EST'] <= end_date)]
    df_30D = data[(data['SESSIONS_DATE_EST'] >= (end_date - pd.to_timedelta(30, unit='d')))&(data['SESSIONS_DATE_EST'] <= end_date)]
    list_of_df = [df_7D,df_14D,df_30D]

    combined_score, all_asset_names, widget_names, zone_name, zone_names = [], [], [], [], []
    widgets = data['WIDGET_NAME'].unique()
    for wdget in widgets:
        zone = data[data['WIDGET_NAME'] == wdget]['ZONE'].iloc[0]
        if zone == None:
            zone_name.append('no-zone')
        else:
            zone_name.append(zone)
    zone_dict = dict(zip(widgets, zone_name))
    
    for dfr in list_of_df:
        #print("Running new df")
        combined = []
        #ctr = 0
        for widget in widgets:
            # ctr = ctr+1
            # if ctr == int(len(widgets)/4): print("25% done")
            # elif ctr == int(len(widgets)/2): print("50% done")
            # elif ctr == int(0.75*len(widgets)): print("75% done")
            # elif ctr == len(widgets): print("100% done")
            #print(widget)
            df = dfr[dfr['WIDGET_NAME'] == widget]

            if len(df['CREATIVE_NAME'].unique()) == 0:
                continue
            else:
                #Calculate mean impressions by assets
                asset_impressions = list(df.groupby(['CREATIVE_NAME'])['ASSET_SEGMENT_VIEWS'].mean())
                asset_impressions = [x / sum(asset_impressions) for x in asset_impressions] #Calculate probability score of impressions by assets
                asset_names = list(df.groupby(['CREATIVE_NAME'])['ASSET_SEGMENT_VIEWS'].mean().index)
                asset_dict = dict(zip(asset_names,asset_impressions))
                #Create probability column to sample assets by probability score
                df['PROB'] = df['CREATIVE_NAME'].map(asset_dict)

                #Resample by REV/K
                df_rev = df[['CREATIVE_NAME','PROB','REV/K']]
                resamples_rev,sol_rev = [],[]
                for i in range(1000):
                    a = df_rev.sample(n=5000, weights='PROB', replace = True)
                    #b = df[['CREATIVE_NAME','PROB','REV/K']]
                    resamples_rev.append(pd.concat([a,df_rev]))

                    resamples_rev[i] = resamples_rev[i].groupby('CREATIVE_NAME').agg({'REV/K':'sum'})
                    resamples_rev[i] = resamples_rev[i].sort_values('REV/K', ascending = False)
                    sol_rev.append(list(resamples_rev[i].index))

                #Calculate sums for each asset and calculate combined metric sums
                sum_rev = []
                unique_assets = df['CREATIVE_NAME'].unique()
                for j in range(len(df['CREATIVE_NAME'].unique())):
                    all_asset_names.append(unique_assets[j])
                    widget_names.append(widget)
                    zone_names.append(zone_dict[widget])
                    sum_rev.append(0)
                    for i in range(1000):
                        sum_rev[j] = sum_rev[j] + sol_rev[i].index(unique_assets[j])+1
                    combined.append(np.round((sum_rev[j])/1000,1))

        combined_score.append(combined)

    assets = pd.DataFrame(list(zip(zone_names,widget_names,all_asset_names,combined_score[0],combined_score[1],combined_score[2])), columns = ['Zone','Widget_Name','Asset_Name','Weekly Combined score','Biweekly Combined score','Monthly Combined score'])
    assets_scored = pd.melt(assets,id_vars=['Zone','Widget_Name','Asset_Name'], value_vars=['Weekly Combined score','Biweekly Combined score','Monthly Combined score'])
    return assets_scored

In [60]:
#asset_scoring()
final_assets = asset_scoring()

In [62]:
final_assets.to_csv('/Users/svijayaku1/Desktop/assets_scored_20.csv')

#### EXTRA

In [ ]:
# def asset_scoring():
#     #Filter by widget, date range and calculate CTR, UNIT/K
#     data = dataset[dataset['CMS_LEVEL'] == 'asset']

#     data['CTR'] = (data['ASSET_SEGMENT_CLICKS']/data['ASSET_SEGMENT_VIEWS']*100).round(2)
#     data['CTR'].replace([np.inf, -np.inf], np.nan, inplace=True)
#     data['CTR'] = data['CTR'].fillna(0)

#     data['UNITS/K'] = ((data['ATTRIBUTED_UNITS']/data['ASSET_SEGMENT_VIEWS'])*1000).round(2)
#     data['UNITS/K'].replace([np.inf, -np.inf], np.nan, inplace=True)
#     data['UNITS/K'] = data['UNITS/K'].fillna(0)

#     end_date = data['SESSIONS_DATE_EST'].max()
#     df_7D = data[(data['SESSIONS_DATE_EST'] >= (end_date - pd.to_timedelta(7, unit='d')))&(data['SESSIONS_DATE_EST'] <= end_date)]
#     df_14D = data[(data['SESSIONS_DATE_EST'] >= (end_date - pd.to_timedelta(14, unit='d')))&(data['SESSIONS_DATE_EST'] <= end_date)]
#     df_30D = data[(data['SESSIONS_DATE_EST'] >= (end_date - pd.to_timedelta(30, unit='d')))&(data['SESSIONS_DATE_EST'] <= end_date)]
#     list_of_df = [df_7D,df_14D,df_30D]

#     combined_score, all_asset_names, widget_names = [], [], []
#     widgets = data['WIDGET_NAME'].unique()
#     for dfr in list_of_df:
#         combined = []
#         for widget in widgets:
#             #print(widget)
#             df = dfr[dfr['WIDGET_NAME'] == widget]

#             #Calculate mean impressions by assets
#             asset_impressions = list(df.groupby(['CREATIVE_NAME'])['ASSET_SEGMENT_VIEWS'].mean())
#             asset_impressions = [x / sum(asset_impressions) for x in asset_impressions] #Calculate probability score of impressions by assets
#             asset_names = list(df.groupby(['CREATIVE_NAME'])['ASSET_SEGMENT_VIEWS'].mean().index)
#             #print(asset_names)
#             asset_dict = dict(zip(asset_names,asset_impressions))
#             #print(asset_dict)
#             #Create probability column to sample assets by probability score
#             df['PROB'] = df['CREATIVE_NAME'].map(asset_dict)

#             #Resample by CTR and UNIT/K
#             df_ctr = df[['CREATIVE_NAME','PROB','CTR']]
#             resamples_ctr = []
#             for i in range(1000):
#                 a = df_ctr.sample(n=5000, weights='PROB', replace = True)
#                 b = df[['CREATIVE_NAME','PROB','CTR']]
#                 resamples_ctr.append(pd.concat([a,b]))
#             sol_ctr = []

#             df_units = df[['CREATIVE_NAME','PROB','UNITS/K']]
#             resamples_units = []
#             for i in range(1000):
#                 a = df_units.sample(n=5000, weights='PROB', replace = True)
#                 b = df[['CREATIVE_NAME','PROB','UNITS/K']]
#                 resamples_units.append(pd.concat([a,b]))   
#             sol_units = []

#             #Sort all 1000 samples for CTR and UNIT/K and append to lists
#             for i in range(1000): 
#                 resamples_ctr[i] = resamples_ctr[i].groupby('CREATIVE_NAME').agg({'CTR':'sum'})
#                 resamples_ctr[i] = resamples_ctr[i].sort_values('CTR', ascending = False)
#                 sol_ctr.append(list(resamples_ctr[i].index)) #list of 1000 lists having asset order

#                 resamples_units[i] = resamples_units[i].groupby('CREATIVE_NAME').agg({'UNITS/K':'sum'})
#                 resamples_units[i] = resamples_units[i].sort_values('UNITS/K', ascending = False)
#                 sol_units.append(list(resamples_units[i].index))

#             #Calculate sums for each asset and calculate combined metric sums
#             sum_ctr,sum_units = [],[]
#             unique_assets = df['CREATIVE_NAME'].unique()
#             for j in range(len(df['CREATIVE_NAME'].unique())):
#                 all_asset_names.append(unique_assets[j])
#                 widget_names.append(widget)
#                 sum_ctr.append(0)
#                 sum_units.append(0)
#                 for i in range(1000):
#                     sum_ctr[j] = sum_ctr[j] + sol_ctr[i].index(unique_assets[j])+1
#                     sum_units[j] = sum_units[j] + sol_units[i].index(unique_assets[j])+1
#                 #Get a combined metric value by weighting ctr - 30% and units/k - 70%
#                 combined.append(np.round(((sum_ctr[j]*0.3)+(sum_units[j]*0.7))/1000,1))

#         combined_score.append(combined)

#     assets = pd.DataFrame(list(zip(widget_names,all_asset_names,combined_score[0],combined_score[1],combined_score[2])), columns = ['Widget_Name','Asset_Name','Weekly Combined score','Biweekly Combined score','Monthly Combined score'])
#     assets_scored = pd.melt(assets,id_vars=['Widget_Name','Asset_Name'], value_vars=['Weekly Combined score','Biweekly Combined score','Monthly Combined score'])
#     return assets_scored